# Internet of Things (2IMN25)
### Machine Learning Assignment

### Introduction

*Add assignment description here*

In [ ]:
""" Import all libraries here """
import numpy as np
import os
import math
from datetime import datetime
import pandas as pd
from dateutil import parser


## Reading the dataset

In the code box below, please read in the data-sets into suitable data structures. The key task here would be to contruct the feature vectors which would be fed to the regression algorithm.

The datasets for the weather, rain and energy consumption have been provided in the zip file, extract this zip file into the same folder as the python notebook. 

*Hint : Be sure to check if there are any missing fields in the provided data*

In [2]:
data_path = os.path.join(os.getcwd(),'data')

""" Read in the weather"""
weather_array = pd.read_csv(os.path.join(data_path,'Weather-Eindhoven.csv'),
                         delimiter = ',',
                         header=0,
                         parse_dates = [0],
                         usecols = [0,2,4,6,7,8]) #Drop "chill", too many missing values
weather_array  = weather_array.as_matrix()

""" Read in the rain data"""
rain_array = pd.read_csv(os.path.join(data_path,'Rain-Best.csv'),
                         delimiter = ',',
                         header=0,
                         parse_dates = [0])

rain_array = rain_array.as_matrix()

""" Synchronize the 2 data-sets by removing the extra data from the rainfall dataset"""
remove_rows = []
for element,idx in zip(rain_array[:,0],range(len(rain_array[:,0]))):
    if element.minute%10 != 0:
        remove_rows.append(idx)

synced_rain_array = np.delete(rain_array,remove_rows,axis = 0)

print(synced_rain_array.shape)
print(weather_array[:,0].shape)

""" Merge the weather and rain data when time-stamps are equal """
merged_array = []
for rainElem in synced_rain_array:
    try :
        merged_row = []
        valid_row = list(weather_array[:,0]).index(rainElem[0])
        for weatherFeature in weather_array[valid_row]:
            merged_row.append(weatherFeature)
        merged_row.append(rainElem[1])
        merged_array.append(np.asarray(merged_row))
    except ValueError:
        print('Did not find matching timestamp in the weather data, continuing')
        continue

merged_array = np.asarray(merged_array)

(43844, 2)
(43872,)
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matchin

In [3]:
""" Clean up the data by removing rows with missing data points """
missing_data_rows = []
for row,idx in zip(merged_array,range(merged_array.shape[0])):
    if row[0].month > 10:
        missing_data_rows.append(idx)
        continue # No energy data for the month of November so delete those too.
    for elem in row[1:]: #Timestamp cannot be "checked"
        if math.isnan(elem):
            missing_data_rows.append(idx)
            break # To prevent the same row from being added multiple times to the list, break when the first "nan" is found
clean_data_array = np.delete(merged_array,missing_data_rows,axis=0)
print(clean_data_array.shape)

(43501, 7)


In [17]:
""" Some helper APIs """
def energy_lookup(timeStamp,dataFolder):
    """ 
    Looks up the energy consumption based on the time-stamp
    Returns energy consumption for the provided time-stamp
    """
    if timeStamp.month <= 9:
        monthFolder = os.path.join(dataFolder,'0{}'.format(timeStamp.month))
    else:
        monthFolder = os.path.join(dataFolder,'{}'.format(timeStamp.month))
    if timeStamp.day <= 9:
        energyFile = os.path.join(monthFolder,'0{}'.format(timeStamp.day))
    else:
        energyFile = os.path.join(monthFolder,'{}'.format(timeStamp.day))
    try :
        energy_array = pd.read_csv(energyFile,
                               delimiter = ',',
                               header=0
                               )
        energy_array = energy_array.as_matrix()
        energy_values = []
    except :
        return None # If date isn't found, add all the rows for deletion
    
    for energy,idx in zip(energy_array[:,3],range(energy_array.shape[0]-1)): # The last line contain "24" in hours. Drop it
        timeObj = parse_time(energy_array[idx][1])
        if timeObj.hour == timeStamp.hour and timeObj.minute == timeStamp.minute:
            energy_values.append(energy)
            break # Exit once you've found the right value of energy
    print('{} {}'.format(timeStamp,energy_values)) #Debug
    if len(energy_values) == 0:
        return None
    else:
        return(np.mean(np.asarray(energy_values)))

def parse_time(timeString):
    """ 
    Takes in a timeString and returns a time object which can 
    be easiy analyzed
    """
    try :
        return parser.parse(timeString).time()
    except ValueError:
        print('{}'.format(timeString))


def create_label_vector(feature_array,labelDataFolder):
    """ Creates a vector of labels corresponding to the feature vector time-stamp """
    labels = []
    missing_label_rows = []
    for featureVector,idx in zip(feature_array,range(feature_array.shape[0])):
        label = energy_lookup(featureVector[0],labelDataFolder)
        if label == None:
            missing_label_rows.append(idx)
        else:
            labels.append(label)           
    return labels,missing_label_rows

labelDataFolder = os.path.join(data_path,'2017')
labels,missing_label_rows = create_label_vector(clean_data_array,labelDataFolder)




2017-01-01 00:00:00 [8158.876]
2017-01-01 00:10:00 [8158.956]
2017-01-01 00:20:00 [8159.019]
2017-01-01 00:30:00 [8159.234]
2017-01-01 00:40:00 [8159.369000000001]
2017-01-01 00:50:00 [8159.479]
2017-01-01 01:00:00 [8159.573]
2017-01-01 01:10:00 [8159.679]
2017-01-01 01:20:00 [8159.77]
2017-01-01 01:30:00 [8159.849]
2017-01-01 01:40:00 [8159.933000000001]
2017-01-01 01:50:00 [8159.991999999999]
2017-01-01 02:00:00 [8160.047]
2017-01-01 02:10:00 [8160.1140000000005]
2017-01-01 02:20:00 [8160.2]
2017-01-01 02:30:00 [8160.295]
2017-01-01 02:40:00 [8160.357]
2017-01-01 02:50:00 [8160.414000000001]
2017-01-01 03:00:00 [8160.48]
2017-01-01 03:10:00 [8160.544]
2017-01-01 03:20:00 [8160.599]
2017-01-01 03:30:00 [8160.651999999999]
2017-01-01 03:40:00 [8160.705]
2017-01-01 03:50:00 [8160.75]
2017-01-01 04:00:00 [8160.785]
2017-01-01 04:10:00 [8160.821]
2017-01-01 04:20:00 [8160.869000000001]
2017-01-01 04:30:00 [8160.905]
2017-01-01 04:40:00 [8160.944]
2017-01-01 04:50:00 [8161.004]
2017-01-01 

2017-01-02 18:00:00 [8174.217]
2017-01-02 18:10:00 [8174.217]
2017-01-02 18:20:00 [8174.217]
2017-01-02 18:30:00 [8174.217]
2017-01-02 18:40:00 [8174.217]
2017-01-02 18:50:00 [8174.217]
2017-01-02 19:00:00 [8174.217]
2017-01-02 19:10:00 [8174.217]
2017-01-02 19:20:00 [8174.217]
2017-01-02 19:30:00 [8174.217]
2017-01-02 19:40:00 [8174.217]
2017-01-02 19:50:00 [8174.217]
2017-01-02 20:00:00 [8174.217]
2017-01-02 20:10:00 [8174.217]
2017-01-02 20:20:00 [8174.217]
2017-01-02 20:30:00 [8174.217]
2017-01-02 20:40:00 [8174.217]
2017-01-02 20:50:00 [8174.217]
2017-01-02 21:00:00 [8174.219]
2017-01-02 21:10:00 [8174.625]
2017-01-02 21:20:00 [8174.731]
2017-01-02 21:30:00 [8174.812]
2017-01-02 21:40:00 [8174.891]
2017-01-02 21:50:00 [8174.984]
2017-01-02 22:00:00 [8175.07]
2017-01-02 22:10:00 [8175.154]
2017-01-02 22:20:00 [8175.318]
2017-01-02 22:30:00 [8175.404]
2017-01-02 22:40:00 [8175.479]
2017-01-02 22:50:00 [8175.559]
2017-01-02 23:00:00 [8175.635]
2017-01-02 23:10:00 [8175.726]
2017-01-0

2017-01-04 14:40:00 [8182.77]
2017-01-04 14:50:00 [8182.77]
2017-01-04 15:00:00 [8182.77]
2017-01-04 15:10:00 [8182.77]
2017-01-04 15:20:00 [8182.77]
2017-01-04 15:30:00 [8182.77]
2017-01-04 15:40:00 [8182.77]
2017-01-04 15:50:00 [8182.77]
2017-01-04 16:00:00 [8182.77]
2017-01-04 16:10:00 [8182.77]
2017-01-04 16:20:00 [8182.77]
2017-01-04 16:30:00 [8182.77]
2017-01-04 16:40:00 [8182.77]
2017-01-04 16:50:00 [8182.77]
2017-01-04 17:00:00 [8182.77]
2017-01-04 17:10:00 [8182.77]
2017-01-04 17:20:00 [8182.77]
2017-01-04 17:30:00 [8182.77]
2017-01-04 17:40:00 [8182.77]
2017-01-04 17:50:00 [8182.77]
2017-01-04 18:00:00 [8182.77]
2017-01-04 18:10:00 [8182.77]
2017-01-04 18:20:00 [8182.77]
2017-01-04 18:30:00 [8182.77]
2017-01-04 18:40:00 [8182.77]
2017-01-04 18:50:00 [8182.77]
2017-01-04 19:00:00 [8182.77]
2017-01-04 19:10:00 [8182.77]
2017-01-04 19:20:00 [8182.77]
2017-01-04 19:30:00 [8182.77]
2017-01-04 19:40:00 [8182.77]
2017-01-04 19:50:00 [8182.77]
2017-01-04 20:00:00 [8182.77]
2017-01-04

2017-01-06 09:40:00 [8190.804]
2017-01-06 09:50:00 [8190.804]
2017-01-06 10:00:00 [8190.804]
2017-01-06 10:10:00 [8190.804]
2017-01-06 10:20:00 [8190.804]
2017-01-06 10:30:00 [8190.804]
2017-01-06 10:40:00 [8190.804]
2017-01-06 10:50:00 [8190.804]
2017-01-06 11:00:00 [8190.804]
2017-01-06 11:10:00 [8190.804]
2017-01-06 11:20:00 [8190.804]
2017-01-06 11:30:00 [8190.804]
2017-01-06 11:40:00 [8190.804]
2017-01-06 11:50:00 [8190.804]
2017-01-06 12:00:00 [8190.804]
2017-01-06 12:10:00 [8190.804]
2017-01-06 12:20:00 [8190.804]
2017-01-06 12:30:00 [8190.804]
2017-01-06 12:40:00 [8190.804]
2017-01-06 12:50:00 [8190.804]
2017-01-06 13:00:00 [8190.804]
2017-01-06 13:10:00 [8190.804]
2017-01-06 13:20:00 [8190.804]
2017-01-06 13:30:00 [8190.804]
2017-01-06 13:40:00 [8190.804]
2017-01-06 13:50:00 [8190.804]
2017-01-06 14:00:00 [8190.804]
2017-01-06 14:10:00 [8190.804]
2017-01-06 14:20:00 [8190.804]
2017-01-06 14:30:00 [8190.804]
2017-01-06 14:40:00 [8190.804]
2017-01-06 14:50:00 [8190.804]
2017-01-

2017-01-08 04:30:00 [8211.519]
2017-01-08 04:40:00 [8211.559000000001]
2017-01-08 04:50:00 [8211.607]
2017-01-08 05:00:00 [8211.665]
2017-01-08 05:10:00 [8211.787]
2017-01-08 05:20:00 [8211.846]
2017-01-08 05:30:00 [8211.893]
2017-01-08 05:40:00 [8211.924]
2017-01-08 05:50:00 [8211.959]
2017-01-08 06:00:00 [8212.001]
2017-01-08 06:10:00 [8212.033000000001]
2017-01-08 06:20:00 [8212.065999999999]
2017-01-08 06:30:00 [8212.109]
2017-01-08 06:40:00 [8212.141]
2017-01-08 06:50:00 [8212.177]
2017-01-08 07:00:00 [8212.224]
2017-01-08 07:10:00 [8212.26]
2017-01-08 07:20:00 [8212.296999999999]
2017-01-08 07:30:00 [8212.344000000001]
2017-01-08 07:40:00 [8212.38]
2017-01-08 07:50:00 [8212.429]
2017-01-08 08:00:00 [8212.484]
2017-01-08 08:10:00 [8212.518]
2017-01-08 08:20:00 [8212.618]
2017-01-08 08:30:00 [8212.665]
2017-01-08 08:40:00 [8212.767]
2017-01-08 08:50:00 [8212.893]
2017-01-08 09:00:00 [8212.957]
2017-01-08 09:10:00 [8213.019]
2017-01-08 09:20:00 [8213.085]
2017-01-08 09:30:00 [8213.1

...

2017-05-31 07:30:00 [9183.396999999999]
2017-05-31 07:40:00 [9183.396999999999]
2017-05-31 07:50:00 [9183.396999999999]
2017-05-31 08:00:00 [9183.396999999999]
2017-05-31 08:10:00 [9183.396999999999]
2017-05-31 08:20:00 [9183.396999999999]
2017-05-31 08:30:00 [9183.396999999999]
2017-05-31 08:40:00 [9183.396999999999]
2017-05-31 08:50:00 [9183.396999999999]
2017-05-31 09:00:00 [9183.396999999999]
2017-05-31 09:10:00 [9183.396999999999]
2017-05-31 09:20:00 [9183.396999999999]
2017-05-31 09:30:00 [9183.396999999999]
2017-05-31 09:40:00 [9183.396999999999]
2017-05-31 09:50:00 [9183.396999999999]
2017-05-31 10:00:00 [9183.396999999999]
2017-05-31 10:10:00 [9183.396999999999]
2017-05-31 10:20:00 [9183.396999999999]
2017-05-31 10:30:00 [9183.396999999999]
2017-05-31 10:40:00 [9183.396999999999]
2017-05-31 10:50:00 [9183.396999999999]
2017-05-31 11:00:00 [9183.396999999999]
2017-05-31 11:10:00 [9183.396999999999]
2017-05-31 11:20:00 [9183.396999999999]
2017-05-31 11:30:00 [9183.396999999999]


2017-06-01 23:20:00 [9190.043]
2017-06-01 23:30:00 [9190.168]
2017-06-01 23:40:00 [9190.314]
2017-06-01 23:50:00 [9190.396999999999]
2017-06-02 00:00:00 [9190.485999999999]
2017-06-02 00:10:00 [9190.56]
2017-06-02 00:20:00 [9190.629]
2017-06-02 00:30:00 [9190.678]
2017-06-02 00:40:00 [9190.73]
2017-06-02 00:50:00 [9190.779]
2017-06-02 01:00:00 [9190.825]
2017-06-02 01:10:00 [9190.879]
2017-06-02 01:20:00 [9190.923]
2017-06-02 01:30:00 [9190.976999999999]
2017-06-02 01:40:00 [9191.038]
2017-06-02 01:50:00 [9191.101999999999]
2017-06-02 02:00:00 [9191.166]
2017-06-02 02:10:00 [9191.223]
2017-06-02 02:20:00 [9191.29]
2017-06-02 02:30:00 [9191.392]
2017-06-02 02:40:00 [9191.440999999999]
2017-06-02 02:50:00 [9191.488000000001]
2017-06-02 03:00:00 [9191.533000000001]
2017-06-02 03:10:00 [9191.585]
2017-06-02 03:20:00 [9191.628]
2017-06-02 03:30:00 [9191.679]
2017-06-02 03:40:00 [9191.724]
2017-06-02 03:50:00 [9191.77]
2017-06-02 04:00:00 [9191.825]
2017-06-02 04:10:00 [9191.869]
2017-06-02 

2017-06-03 14:20:00 [9199.654]
2017-06-03 14:30:00 [9199.654]
2017-06-03 14:40:00 [9199.654]
2017-06-03 14:50:00 [9199.654]
2017-06-03 15:00:00 [9199.654]
2017-06-03 15:10:00 [9199.654]
2017-06-03 15:20:00 [9199.655]
2017-06-03 15:30:00 [9199.702]
2017-06-03 15:40:00 [9199.703000000001]
2017-06-03 15:50:00 [9199.722]
2017-06-03 16:00:00 [9199.767]
2017-06-03 16:10:00 [9199.801]
2017-06-03 16:20:00 [9199.803]
2017-06-03 16:30:00 [9199.804]
2017-06-03 16:40:00 [9199.805]
2017-06-03 16:50:00 [9199.875]
2017-06-03 17:00:00 [9199.898000000001]
2017-06-03 17:10:00 [9199.942]
2017-06-03 17:20:00 [9199.951]
2017-06-03 17:30:00 [9199.954]
2017-06-03 17:40:00 [9199.954]
2017-06-03 17:50:00 [9199.955]
2017-06-03 18:00:00 [9199.961]
2017-06-03 18:10:00 [9199.966]
2017-06-03 18:20:00 [9199.973]
2017-06-03 18:30:00 [9200.15]
2017-06-03 18:40:00 [9200.586]
2017-06-03 18:50:00 [9201.189]
2017-06-03 19:00:00 [9201.747]
2017-06-03 19:10:00 [9202.226]
2017-06-03 19:20:00 [9202.651]
2017-06-03 19:30:00 [9

2017-06-05 08:20:00 [9218.212]
2017-06-05 08:30:00 [9218.213]
2017-06-05 08:40:00 [9218.213]
2017-06-05 08:50:00 [9218.213]
2017-06-05 09:00:00 [9218.254]
2017-06-05 09:10:00 [9218.254]
2017-06-05 09:20:00 [9218.254]
2017-06-05 09:30:00 [9218.254]
2017-06-05 09:40:00 [9218.254]
2017-06-05 09:50:00 [9218.254]
2017-06-05 10:00:00 [9218.254]
2017-06-05 10:10:00 [9218.254]
2017-06-05 10:20:00 [9218.254]
2017-06-05 10:30:00 [9218.254]
2017-06-05 10:40:00 [9218.254]
2017-06-05 10:50:00 [9218.254]
2017-06-05 11:00:00 [9218.268]
2017-06-05 11:10:00 [9218.280999999999]
2017-06-05 11:20:00 [9218.280999999999]
2017-06-05 11:30:00 [9218.280999999999]
2017-06-05 11:40:00 [9218.280999999999]
2017-06-05 11:50:00 [9218.280999999999]
2017-06-05 12:00:00 [9218.280999999999]
2017-06-05 12:10:00 [9218.33]
2017-06-05 12:20:00 [9218.363000000001]
2017-06-05 12:30:00 [9218.415]
2017-06-05 12:40:00 [9218.416]
2017-06-05 12:50:00 [9218.416]
2017-06-05 13:00:00 [9218.416]
2017-06-05 13:10:00 [9218.416]
2017-06-

KeyboardInterrupt: 

NameError: name 'missing_label_rows' is not defined